# 数据源与参考集构建
锚点序列来源于 Spider Silkome Database (https://spider-silkome.org/about)。
## 环境配置
确保已安装以下依赖 (Via Pixi):
```bash
pixi init spider_silkome
pixi add python pandas biopython mafft hmmer samtools flye minimap2 seqkit cd-hit
```

In [ ]:
import os
import subprocess
import glob
import pandas as pd
from spider_silkome_module.config import PROCESSED_DATA_DIR, RAW_DATA_DIR, SCRIPTS_DIR, EXTERNAL_DATA_DIR, INTERIM_DATA_DIR

from spider_silkome_module import run_shell_command_with_check
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass

# Global Configuration
class Config:
    """
    Configuration class to hold paths and thresholds.
    Using a centralized config makes modifications easier.
    """
    PROJECT_NAME = "spider_silkome_20251222"
    # Paths
    SEEDS_QC_DIR = INTERIM_DATA_DIR / PROJECT_NAME / "spidroin_seeds_qc"
    HMMBUILD_INPUT_DIR = INTERIM_DATA_DIR / PROJECT_NAME / "hmmbuild_input"
    HMMBUILD_OUTPUT_DIR = INTERIM_DATA_DIR / PROJECT_NAME / "hmmbuild_output"
    HMMSEARCH_OUTPUT_DIR = INTERIM_DATA_DIR / PROJECT_NAME / "hmmserch_output"
    MINIPROT_OUTPUT_DIR = INTERIM_DATA_DIR / PROJECT_NAME / "miniprot_output"
    spider_genome_path = RAW_DATA_DIR / "spider_genome"
    spider_genomes = ["Araneus_ventricosus",
                    "Pardosa_pseudoannulata",
                    "Evarcha_sp",
                    "Pholcus_sp",
                    "Heteropoda_venatoria",
                    "Scorpiops_zhui",
                    "Hippasa_lycosina",
                    "Songthela_sp",
                    "Pandercetes_sp",
                    "Trichonephila_clavata"
    ]



    # Threads
    THREADS = 8

    # Thresholds
    E_VALUE_THRES = 1e-10
    MIN_GENE_LEN = 500       # Minimum distance between NTD and CTD
    MAX_GENE_LEN = 100000    # Maximum distance (100kb)

    def __init__(self):
        # Create directories if they don't exist
        os.makedirs(Config.SEEDS_QC_DIR, exist_ok=True)
        os.makedirs(Config.HMMBUILD_INPUT_DIR, exist_ok=True)
        os.makedirs(Config.HMMBUILD_OUTPUT_DIR, exist_ok=True)
        os.makedirs(Config.HMMSEARCH_OUTPUT_DIR, exist_ok=True)
        os.makedirs(Config.MINIPROT_OUTPUT_DIR, exist_ok=True)

# Initialize configuration
config = Config()

spidroin_seeds = EXTERNAL_DATA_DIR / "spidroin_seeds_collection.faa"

## 1. 种子序列的获取与清洗
蛛丝蛋白家族庞大，涵盖了大壶状腺丝（MaSp）、小壶状腺丝（MiSp）、鞭毛状腺丝（Flag）、葡萄状腺丝（AcSp）、梨状腺丝（PySp）、管状腺丝（TuSp/CySp）以及聚合腺丝（AgSp）等多种类型。此外，Schöneberg等人（2025）还报道了在中纺亚目中发现的独特spidroin类型。
操作步骤：
1. 下载分类： 从数据库中分别下载各亚家族的NTD和CTD氨基酸序列。
2. 质量控制： 剔除序列中简单的重复序列或长度异常（<50aa）的序列。
3. 去冗余： 使用CD-HIT工具在95%的相似度水平上去除高度重复的序列，以防止模型过拟合。

In [ ]:
# 质量控制
qc_cmd = f"pixi run python3 {SCRIPTS_DIR}/extract_terminal_domains.py {spidroin_seeds} -o {config.SEEDS_QC_DIR} -l 50 --similarity 0.5"
run_shell_command_with_check(qc_cmd, config.SEEDS_QC_DIR/"processing_report.tsv", force=True)

In [ ]:
# 使用cd-hit去除高度重复的序列
for input_path in Config.SEEDS_QC_DIR.glob("*TD.faa"):
    output_path = input_path.with_stem(f"{input_path.stem}_de_dup")
    de_dup_cmd = f"pixi run cd-hit -i {input_path} -o {output_path} -c 0.95 -T 0 -M 0"
    run_shell_command_with_check(de_dup_cmd, output_path)

## 2 多序列比对与pHMM构建
为了提高搜索的灵敏度，特别是针对远缘物种（如从圆网蛛到中纺亚目蜘蛛的跨越），必须构建Profile HMM。

- 技术细节：
  - 工具选择： HMMER 3.4 suite (hmmbuild)。
  - 比对策略： 使用 MAFFT 的 L-INS-i 算法进行多序列比对（MSA）。该算法对捕捉局部结构特征（如NTD中的保守半胱氨酸残基或特定的α-螺旋结构）最为准确。

- 模型训练：
  - 构建亚家族特异性模型（如 MaSp_NTD.hmm）：用于精准分类。

- 校准： 使用 hmmpress 对模型进行二进制压缩与索引。

In [ ]:
# 比对
for input_path in Config.SEEDS_QC_DIR.glob("*_de_dup.faa"):
    name = input_path.stem.split("_de")[0]
    mafft_output_path = Config.HMMBUILD_INPUT_DIR / f"{name}_aln.faa"
    mafft_cmd = f"mafft --maxiterate 1000 --localpair --thread -1 {input_path} > {mafft_output_path}"
    run_shell_command_with_check(mafft_cmd, mafft_output_path)


In [ ]:
# 裁剪
for input_path in Config.HMMBUILD_INPUT_DIR.glob("*_aln.faa"):
    aln_trimed_output_path = input_path.with_name(input_path.stem + "_trimed.faa")
    aln_trimed_cmd = f"pixi run trimal -in {input_path} -out {aln_trimed_output_path} -gt 0.8 -cons 60"
    run_shell_command_with_check(aln_trimed_cmd, aln_trimed_output_path)

In [ ]:
# 构建 hmm 模型
for input_path in Config.HMMBUILD_INPUT_DIR.glob("*_aln_trimed.faa"):
    name = input_path.stem.split("_aln")[0]
    hmmbuild_output_path = Config.HMMBUILD_OUTPUT_DIR / f"{name}.hmm"
    hmmbuild_cmd = f"hmmbuild -n {name} --amino --cpu 70 {hmmbuild_output_path} {input_path}"
    run_shell_command_with_check(hmmbuild_cmd, hmmbuild_output_path)
    hmmpress_cmd = f"hmmpress {hmmbuild_output_path}"
    run_shell_command_with_check(hmmpress_cmd, f"{hmmbuild_output_path}.h3m")


## 3 基因组扫描策略
传统的BLAST搜索在处理高分歧序列时往往会产生大量假阴性。相比之下，nhmmer（HMMER组件）可以直接将蛋白的pHMM模型映射到核酸序列上，或者使用DNA-to-DNA的HMM搜索，从而在保持高特异性的同时极大提升灵敏度。

**执行方案：**

- 全基因组扫描： 使用 nhmmer 对目标基因组进行六框翻译搜索（或直接核酸搜索，取决于模型类型）。
- 参数设定： E-value 阈值设为 $1e^{-10}$ 以保证高置信度；--qcov（查询覆盖度）需大于70%，确保找到的是完整的结构域而非碎片。
- 结果过滤： 剔除低复杂度的非特异性匹配。蛛丝蛋白基因组中常含有大量的简单重复序列（Simple Repeats），需利用RepeatMasker的输出结果对HMM hits进行软过滤。

In [ ]:
# 共有序列提取
for input_path in Config.HMMBUILD_OUTPUT_DIR.glob("*.hmm"):
    output_path = input_path.with_name(input_path.stem + "_consensus.fa")
    cmd = f"pixi run hmmemit -c {input_path} > {output_path}"
    run_shell_command_with_check(cmd, output_path)

In [ ]:
# miniprot search
for spider in Config.spider_genomes:
    spider_genome = Config.spider_genome_path / spider / f"{spider}.fa"
    spider_genome_index = spider_genome.with_name(spider + ".mpi")
    index_cmd = f"pixi run miniprot -t70 -d  {spider_genome_index} {spider_genome}"
    run_shell_command_with_check(index_cmd, spider_genome_index)
    for consensus_seq in Config.HMMBUILD_OUTPUT_DIR.glob("*_consensus.fa"):
        output = Config.MINIPROT_OUTPUT_DIR / spider / f"{consensus_seq.stem}.gff"
        miniprot_cmd = f"pixi run miniprot -t 70 -I --gff {spider_genome_index} {consensus_seq} > {output}"
        run_shell_command_with_check(miniprot_cmd, output)